In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import pandas as pd

import os

from datetime import datetime, timedelta, date


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#### Setting up strava API client

#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

## Enable accessing private activities
auth_url = client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)

client = stravalib.client.Client(access_token=access_token)
athlete = client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))

athlete name Brian, athlete id 3360678.


In [3]:
### Setting up google sheets client -- old library
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials


# # use creds to create a client to interact with the Google Drive API
# scope = ['https://spreadsheets.google.com/feeds']
# creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
# gspread_client = gspread.authorize(creds)
 


In [4]:
## Set up google sheets client, open worksheet
import pygsheets

gc = pygsheets.authorize(outh_file='client_secret.json', no_cache=True)

# Open spreadsheet and then workseet
sh = gc.open('Milburn Ultimate Scores')
wks = sh.sheet1

In [5]:
## Get last entry from Data Spreadsheet

dates_recorded = [datetime.strptime(d, '%Y-%m-%d') for d in wks.get_col(1) if d != '' and d != 'Date']
lap_start_date = max(dates_recorded) + timedelta(days=1)
dates_recorded, lap_start_date

([datetime.datetime(2017, 10, 10, 0, 0)],
 datetime.datetime(2017, 10, 11, 0, 0))

In [6]:
runs = []
for activity in client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        runs.append(activity)

In [7]:
def get_strava_description(activity):
    new_activity = client.get_activity(activity.id)
    try:
        scores, color = new_activity.description.split(' ')
        try:
            team_score, opponent_score = scores.split('-')
        except ValueError:
            team_score, opponent_score, color = None, None, None     
    except ValueError:
        scores = None
        team_score, opponent_score, color = None, None, None           

    try:
        color = color.lower()
    except AttributeError:
        color = None
        
    return team_score, opponent_score, color

In [8]:
def extract_events(run):
    lap_nums = []
    start_times = []
    elapsed_times = []
    for l in run.laps:
        lap_nums.append(int(l.name.split(' ')[-1]))
        start_times.append(l.start_date_local)
        elapsed_times.append(l.elapsed_time)

    lap_nums = np.array(lap_nums)
    
    events = []
    for n, s, e in zip(np.diff(lap_nums), start_times, elapsed_times):
        events.append([n, s, e])
    
    return (events)
        
def process_events(events):
    
    event_lookup = [
        '',
        'team_point',
        'opponent_point',
        'my_point',
        'game',
        'game'
    ]
    
    games = []
    game_num = 0
    added = False

    base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0, 'events':[], 'start_time':None, 'end_time':None}
    game = base_game

    for event in events:
        if game['start_time'] is None:
            game['start_time'] = event[1]

        game['end_time'] = event[1] + event[2]

        event_type = event_lookup[event[0]]
        game['events'].append((event_type, event[1], event[2]))


        if event_type == 'game':
            games.append(game)
            game_num += 1
            game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num, 'events':[], 'start_time':None, 'end_time':None}
            added = True

        elif event_type == 'my_point':
            game[event_type] += 1
            game['team_point'] += 1
            added = False

        else:
            game[event_type] += 1
            added = False


    if not added:
        games.append(game)
    
    
    ## Assign game winners
    for game in games:
        if game['team_point'] > game['opponent_point']:
            game['win'] = True
        else:
            game['win'] = False
            
    return games

In [9]:
games = []
for run in runs:
    team_score, opponent_score, color = get_strava_description(run)
    print(run.start_date_local, get_strava_description(run))
    events = extract_events(run)
    current_games = process_events(events)
    for g in current_games:
        g['my_color'] = color

    games = games + current_games
    

2017-10-11 12:17:28 ('1', '2', 'colors')
2017-10-13 12:11:22 ('1', '3', 'white')
2017-10-16 12:13:20 ('1', '2', 'colors')
2017-10-18 12:10:52 ('1', '2', 'white')
2017-10-20 12:16:14 ('2', '0', 'white')
2017-10-23 12:11:10 ('0', '4', 'white')
2017-10-25 12:13:00 ('1', '2', 'colors')
2017-10-27 12:57:16 (None, None, None)
2017-10-30 12:15:12 ('0', '2', 'colors')
2017-11-01 12:11:39 ('1', '3', 'white')
2017-11-03 12:07:24 ('1', '3', 'colors')
2017-11-06 12:09:48 ('2', '1', 'white')
2017-11-08 12:17:26 ('0', '3', 'colors')
2017-11-15 12:07:22 ('3', '1', 'white')
2017-11-17 12:16:55 ('2', '0', 'colors')
2017-11-20 12:12:23 ('2', '1', 'white')
2017-11-22 12:14:53 (None, None, None)
2017-11-27 12:12:54 ('1', '2', 'white')
2017-11-29 12:14:38 ('3', '1', 'colors')
2017-12-01 12:14:16 ('3', '1', 'white')
2017-12-06 12:04:49 (None, None, None)
2017-12-08 12:10:57 ('3', '1', 'white')
2017-12-11 12:12:01 ('4', '0', 'white')
2017-12-13 12:10:29 ('1', '3', 'white')
2017-12-15 12:16:30 ('3', '0', 'whi

In [13]:
df = pd.DataFrame(games).dropna()

In [14]:
df['date'] = df.end_time.apply(lambda x: date(x.year, x.month, x.day))

df = df.set_index(['date', 'game_num'], drop=False)
df

end_time  \
date       game_num                       
2017-10-11 0        2017-10-11 12:36:34   
           1        2017-10-11 13:00:44   
2017-10-13 0        2017-10-13 12:29:02   
           1        2017-10-13 12:51:36   
           2        2017-10-13 13:03:46   
           3        2017-10-13 13:16:07   
2017-10-16 0        2017-10-16 12:30:07   
           1        2017-10-16 12:45:05   
           2        2017-10-16 12:57:19   
2017-10-18 0        2017-10-18 12:29:00   
           1        2017-10-18 12:47:07   
           2        2017-10-18 13:04:35   
2017-10-20 0        2017-10-20 12:29:34   
           1        2017-10-20 13:05:17   
2017-10-23 0        2017-10-23 12:22:45   
           1        2017-10-23 12:35:35   
           2        2017-10-23 12:56:52   
2017-10-25 0        2017-10-25 12:32:08   
           1        2017-10-25 12:47:59   
           2        2017-10-25 13:04:23   
2017-10-30 0        2017-10-30 12:37:23   
           1        2017-10-30 13:02:08   
2017-11-01 0        2017-11-01 12:24:40   
           1        2017-11-01 12:51:57   
           2        2017-11-01 12:59:56   
2017-11-03 0        2017-11-03 12:25:22   
           1        2017-11-03 12:39:13   
           2        2017-11-03 12:55:19   
           3        2017-11-03 13:10:37   
2017-11-06 0        2017-11-06 12:27:50   
...                                 ...   
2017-11-29 0        2017-11-29 12:27:39   
           1        2017-11-29 12:41:27   
           2        2017-11-29 13:02:48   
           3        2017-11-29 13:10:18   
2017-12-01 0        2017-12-01 12:24:47   
           1        2017-12-01 13:02:52   
           2        2017-12-01 13:12:55   
2017-12-08 0        2017-12-08 12:29:56   
           1        2017-12-08 12:48:28   
           2        2017-12-08 13:03:20   
           3        2017-12-08 13:16:33   
2017-12-11 0        2017-12-11 12:25:54   
           1        2017-12-11 12:42:08   
           2        2017-12-11 13:06:44   
2017-12-13 0        2017-12-13 12:24:08   
           1        2017-12-13 12:44:49   
           2        2017-12-13 13:01:34   
           3        2017-12-13 13:07:12   
2017-12-15 0        2017-12-15 12:35:45   
           1        2017-12-15 12:51:07   
           2        2017-12-15 13:17:12   
2017-12-18 0        2017-12-18 12:29:15   
           1        2017-12-18 12:53:50   
           2        2017-12-18 13:08:48   
           3        2017-12-18 13:23:36   
2017-12-20 0        2017-12-20 12:47:23   
           1        2017-12-20 13:04:51   
           2        2017-12-20 13:14:34   
2018-01-03 0        2018-01-03 12:47:52   
           1        2018-01-03 13:07:58   

                                                                events  \
date       game_num                                                      
2017-10-11 0                    [(game, 2017-10-11 12:17:28, 0:19:06)]   
           1         [(team_point, 2017-10-11 12:36:43, 0:02:25), (...   
2017-10-13 0         [(team_point, 2017-10-13 12:11:22, 0:04:51), (...   
           1         [(team_point, 2017-10-13 12:29:11, 0:02:08), (...   
           2         [(team_point, 2017-10-13 12:51:39, 0:02:20), (...   
           3         [(team_point, 2017-10-13 13:03:50, 0:02:08), (...   
2017-10-16 0         [(opponent_point, 2017-10-16 12:13:20, 0:02:39...   
           1         [(opponent_point, 2017-10-16 12:30:14, 0:01:22...   
           2         [(team_point, 2017-10-16 12:45:15, 0:02:06), (...   
2017-10-18 0         [(opponent_point, 2017-10-18 12:10:52, 0:01:16...   
           1         [(opponent_point, 2017-10-18 12:29:03, 0:02:47...   
           2         [(opponent_point, 2017-10-18 12:47:07, 0:02:10...   
2017-10-20 0         [(team_point, 2017-10-20 12:16:14, 0:00:29), (...   
           1         [(opponent_point, 2017-10-20 12:29:38, 0:02:00...   
2017-10-23 0         [(my_point, 2017-10-23 12:11:10, 0:00:03), (op...   
           1         [(opponent_point, 2017-10-23 12:22:46, 0:0

In [15]:
pdf = df

In [16]:
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None
pdf['game_winner'] = None

In [17]:
for (date, game_num), row in pdf.iterrows():      
    
    if row.my_color == 'white':
        if row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'white_wins'] = team_score                
        pdf.loc[(date, game_num), 'color_wins'] = opponent_score   
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'opponent_point']

    else:
        if not row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'color_wins'] = team_score                
        pdf.loc[(date, game_num), 'white_wins'] = opponent_score   
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'opponent_point']


In [18]:
pdf

end_time  \
date       game_num                       
2017-10-11 0        2017-10-11 12:36:34   
           1        2017-10-11 13:00:44   
2017-10-13 0        2017-10-13 12:29:02   
           1        2017-10-13 12:51:36   
           2        2017-10-13 13:03:46   
           3        2017-10-13 13:16:07   
2017-10-16 0        2017-10-16 12:30:07   
           1        2017-10-16 12:45:05   
           2        2017-10-16 12:57:19   
2017-10-18 0        2017-10-18 12:29:00   
           1        2017-10-18 12:47:07   
           2        2017-10-18 13:04:35   
2017-10-20 0        2017-10-20 12:29:34   
           1        2017-10-20 13:05:17   
2017-10-23 0        2017-10-23 12:22:45   
           1        2017-10-23 12:35:35   
           2        2017-10-23 12:56:52   
2017-10-25 0        2017-10-25 12:32:08   
           1        2017-10-25 12:47:59   
           2        2017-10-25 13:04:23   
2017-10-30 0        2017-10-30 12:37:23   
           1        2017-10-30 13:02:08   
2017-11-01 0        2017-11-01 12:24:40   
           1        2017-11-01 12:51:57   
           2        2017-11-01 12:59:56   
2017-11-03 0        2017-11-03 12:25:22   
           1        2017-11-03 12:39:13   
           2        2017-11-03 12:55:19   
           3        2017-11-03 13:10:37   
2017-11-06 0        2017-11-06 12:27:50   
...                                 ...   
2017-11-29 0        2017-11-29 12:27:39   
           1        2017-11-29 12:41:27   
           2        2017-11-29 13:02:48   
           3        2017-11-29 13:10:18   
2017-12-01 0        2017-12-01 12:24:47   
           1        2017-12-01 13:02:52   
           2        2017-12-01 13:12:55   
2017-12-08 0        2017-12-08 12:29:56   
           1        2017-12-08 12:48:28   
           2        2017-12-08 13:03:20   
           3        2017-12-08 13:16:33   
2017-12-11 0        2017-12-11 12:25:54   
           1        2017-12-11 12:42:08   
           2        2017-12-11 13:06:44   
2017-12-13 0        2017-12-13 12:24:08   
           1        2017-12-13 12:44:49   
           2        2017-12-13 13:01:34   
           3        2017-12-13 13:07:12   
2017-12-15 0        2017-12-15 12:35:45   
           1        2017-12-15 12:51:07   
           2        2017-12-15 13:17:12   
2017-12-18 0        2017-12-18 12:29:15   
           1        2017-12-18 12:53:50   
           2        2017-12-18 13:08:48   
           3        2017-12-18 13:23:36   
2017-12-20 0        2017-12-20 12:47:23   
           1        2017-12-20 13:04:51   
           2        2017-12-20 13:14:34   
2018-01-03 0        2018-01-03 12:47:52   
           1        2018-01-03 13:07:58   

                                                                events  \
date       game_num                                                      
2017-10-11 0                    [(game, 2017-10-11 12:17:28, 0:19:06)]   
           1         [(team_point, 2017-10-11 12:36:43, 0:02:25), (...   
2017-10-13 0         [(team_point, 2017-10-13 12:11:22, 0:04:51), (...   
           1         [(team_point, 2017-10-13 12:29:11, 0:02:08), (...   
           2         [(team_point, 2017-10-13 12:51:39, 0:02:20), (...   
           3         [(team_point, 2017-10-13 13:03:50, 0:02:08), (...   
2017-10-16 0         [(opponent_point, 2017-10-16 12:13:20, 0:02:39...   
           1         [(opponent_point, 2017-10-16 12:30:14, 0:01:22...   
           2         [(team_point, 2017-10-16 12:45:15, 0:02:06), (...   
2017-10-18 0         [(opponent_point, 2017-10-18 12:10:52, 0:01:16...   
           1         [(opponent_point, 2017-10-18 12:29:03, 0:02:47...   
           2         [(opponent_point, 2017-10-18 12:47:07, 0:02:10...   
2017-10-20 0         [(team_point, 2017-10-20 12:16:14, 0:00:29), (...   
           1         [(opponent_point, 2017-10-20 12:29:38, 0:02:00...   
2017-10-23 0         [(my_point, 2017-10-23 12:11:10, 0:00:03), (op...   
           1         [(opponent_point, 2017-10-23 12:22:46, 0:0

In [19]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [20]:
scores = []
for (date, game_num), game in pdf.iterrows():
    base_dict = dict(date=date, game_num=game_num, white_wins=game.white_wins, color_wins=game.color_wins, game_winner=game.game_winner)    
    scores.append(merge_two_dicts(base_dict, dict(team='white', team_score=game.white_point, my_score=game.my_point if game.my_color == 'white' else None)))
    scores.append(merge_two_dicts(base_dict, dict(team='color', team_score=game.color_point, my_score=game.my_point if game.my_color == 'colors' else None)))

score_df = pd.DataFrame(scores).set_index(['date', 'game_num', 'team'], drop=False)

In [21]:
out_df = score_df[['date', 'white_wins', 'color_wins', 'game_num', 'game_winner', 'team', 'team_score', 'my_score']].sort_index(ascending=False, level=0).fillna(value='')
out_df['date'] = out_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [22]:
data = out_df.as_matrix().tolist()
wks.insert_rows(2, values=data, number=len(data))

In [23]:
out_df

date white_wins color_wins  game_num  \
date       game_num team                                                
2018-01-03 1        white  2018-01-03          2          1         1   
                    color  2018-01-03          2          1         1   
           0        white  2018-01-03          2          1         0   
                    color  2018-01-03          2          1         0   
2017-12-20 2        white  2017-12-20          2          1         2   
                    color  2017-12-20          2          1         2   
           1        white  2017-12-20          2          1         1   
                    color  2017-12-20          2          1         1   
           0        white  2017-12-20          2          1         0   
                    color  2017-12-20          2          1         0   
2017-12-18 3        white  2017-12-18          1          2         3   
                    color  2017-12-18          1          2         3   
           2        white  2017-12-18          1          2         2   
                    color  2017-12-18          1          2         2   
           1        white  2017-12-18          1          2         1   
                    color  2017-12-18          1          2         1   
           0        white  2017-12-18          1          2         0   
                    color  2017-12-18          1          2         0   
2017-12-15 2        white  2017-12-15          2          1         2   
                    color  2017-12-15          2          1         2   
           1        white  2017-12-15          2          1         1   
                    color  2017-12-15          2          1         1   
           0        white  2017-12-15          2          1         0   
                    color  2017-12-15          2          1         0   
2017-12-13 3        white  2017-12-13          2          1         3   
                    color  2017-12-13          2          1         3   
           2        white  2017-12-13          2          1         2   
                    color  2017-12-13          2          1         2   
           1        white  2017-12-13          2          1         1   
                    color  2017-12-13          2          1         1   
...                               ...        ...        ...       ...   
2017-10-23 0        white  2017-10-23          2          1         0   
                    color  2017-10-23          2          1         0   
2017-10-20 1        white  2017-10-20          2          1         1   
                    color  2017-10-20          2          1         1   
           0        white  2017-10-20          2          1         0   
                    color  2017-10-20          2          1         0   
2017-10-18 2        white  2017-10-18          2          1         2   
                    color  2017-10-18          2          1         2   
           1        white  2017-10-18          2          1         1   
                    color  2017-10-18          2          1         1   
           0        white  2017-10-18          2          1         0   
                    color  2017-10-18          2          1         0   
2017-10-16 2        white  2017-10-16          1          2         2   
                    color  2017-10-16          1          2         2   
           1        white  2017-10-16          1          2         1   
                    color  2017-10-16          1          2         1   
           0        white  2017-10-16          1          2         0   
                    color  2017-10-16          1          2         0   
2017-10-13 3        white  2017-10-13          2          1         3   
                    color  2017-10-13          2          1         3   
           2        white  2017-10-13          2          1         2   
                    color  2017-10-13          2          1         2   
           1      